In [1]:

# just to make the cells appear wider:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



import h5py as h5
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import string


/var/folders/lc/vg4hc2nn2pn05y4lw3h5wmpm0000gn/T/ipykernel_11944/3036708214.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# 1 Download and read in the BHNS datafile:


1A. Download the BHNS file that you want from the Publicly available Zenodo: https://zenodo.org/record/5178777 
The example below shows the output for model 'A', which you can obtain by downloading the fiducial.zip file. 


all lines of code that you might have to change are given with "# change this line! " 

In [7]:
# to obtain properties of ALL binaries simulated, do this:

DCOtype = 'BHNS'   # You can change this line to 'BBH', 'BHNS' 'BNS', or 'ALL' (All DCOs)  # change this line! (but required downloading BBH/BNS data from Zenodo)


# add path to where the COMPASOutput.h5 file is stored. 
# For you the part '/Volumes/Andromeda2/DATA/AllDCO_bugfix/' is probably different
path = '/Volumes/Andromeda2/DATA/AllDCO_bugfix/fiducial/COMPASCompactOutput_'+ DCOtype +'_A.h5' # change this line! 


print('excecuting this code might take a little while (~few min) \n')
fdata = h5.File(path)
# shows the different files within the hdf5 folder 

print('the available datasets for this file are:')
print(fdata.keys())










excecuting this code might take a little while (~few min) 

the available datasets for this file are:
<KeysViewHDF5 ['RLOF', 'commonEnvelopes', 'doubleCompactObjects', 'formationChannels', 'supernovae', 'systems', 'weights_detected', 'weights_detectedPerRedshift', 'weights_intrinsic', 'weights_intrinsicPerRedshift']>


### M1, M2 and Chirpmass in Msun

The most used parameters are quoted in the file "doubleCompactObjects", that describes many properties of the binaries that form the type of double-compact object (DCO)
merger (here, BHNS) below is an example: 


In [8]:
# see several parameters that are contained in this file
print(fdata['doubleCompactObjects'].keys())

<KeysViewHDF5 ['COCoreMassDCOFormation1', 'COCoreMassDCOFormation2', 'ECSNPrimary', 'ECSNSecondary', 'HeCoreMassDCOFormation1', 'HeCoreMassDCOFormation2', 'ID', 'M1', 'M1ZAMS', 'M2', 'M2ZAMS', 'Metallicity1', 'Metallicity2', 'PISNPrimary', 'PISNSecondary', 'PPISNPrimary', 'PPISNSecondary', 'PrimaryMTCase', 'RL1to2PostCEE', 'RL1to2PreCEE', 'RL2to1PostCEE', 'RL2to1PreCEE', 'RLOFSecondaryAfterCEE', 'SecondaryMTCase', 'SemiMajorAxisPostCEE', 'SemiMajorAxisPreCEE', 'USSNPrimary', 'USSNSecondary', 'coreMassDCOFormation1', 'coreMassDCOFormation2', 'doubleCommonEnvelopeFlag', 'drawnKick1', 'drawnKick2', 'eccentricityDCOFormation', 'eccentricityInitial', 'eccentricityPrior2ndSN', 'kickDirectionPower', 'mergesInHubbleTimeFlag', 'optimisticCEFlag', 'phiSupernova1', 'phiSupernova2', 'recycledPrimary', 'recycledSecondary', 'relativeVelocity2ndSN', 'samplingPhase', 'seed', 'separationDCOFormation', 'separationInitial', 'separationPrior2ndSN', 'sigmaKickBH', 'sigmaKickNS', 'stellarType1', 'stellarTyp

The weights e.g. for LIGO weighted are also given in these files, under "weights_detected", this takes into account the star formation history of all mergers and the sensitivity of a GW detector at design configuration (LVK). 



In [9]:
print(fdata['weights_detected'].keys()) 




<KeysViewHDF5 ['SEED', 'w_000', 'w_111', 'w_112', 'w_113', 'w_121', 'w_122', 'w_123', 'w_131', 'w_132', 'w_133', 'w_211', 'w_212', 'w_213', 'w_221', 'w_222', 'w_223', 'w_231', 'w_232', 'w_233', 'w_311', 'w_312', 'w_313', 'w_321', 'w_322', 'w_323', 'w_331', 'w_332', 'w_333']>


In [ ]:
# to obtain the properties of the selected DCOtype you simply do this:

fDCO      = fdata['doubleCompactObjects']
#Stuff I need for cosmological integral


# at this moment we dont need to specify a mask, since our datafile has already taken this into account. 
DCOmask = [True]*(len(fDCO['Metallicity1'][...].squeeze()))
print(DCOmask)


metallicitySystems  = fDCO['Metallicity1'][...].squeeze()[DCOmask]  # Metallicity at ZAMS 
delayTimes          = fDCO['tform'][...].squeeze()[DCOmask] + \
                           fDCO['tc'][...].squeeze()[DCOmask]   # delay time 
tc                  = fDCO['tc'][...].squeeze()[DCOmask]  # coalescence time (or merger time)
M1              = fDCO['M1'][...].squeeze()[DCOmask]     # Compact object mass of star 1 
M2               = fDCO['M2'][...].squeeze()[DCOmask]    # Compact object mass of star 2 
m1zams              = fDCO['M1ZAMS'][...].squeeze()[DCOmask]   # Mass at ZAMS of star 1 
m2zams              = fDCO['M2ZAMS'][...].squeeze()[DCOmask]   # Mass at ZAMS of star 2 
separationzams      = fDCO['separationInitial'][...].squeeze()[DCOmask]   # separation at ZAMS of binary 


# we will use for this demo the weights from Star formation history model xyz = '000', these can be obtained using:
weights = fdata['weights_detected']['w_000']

# change this to formation weights by uncommenting the following line:
# weights = fDCO['weight']


# other models can be chosen too. (there are 28 options currently)


def chirpmass(m1, m2):
    numer = (m1*m2)**(3./5)
    denom = (m1+m2)**(1./5)
    
    return numer/denom

# and you can plot properties, e.g., the chirpmass distribution: 
chirpmass =chirpmass(m1=M1, m2=M2)


plt.hist(chirpmass, bins=100, weights=weights)
plt.xlabel('chirpmass [Msun]')
plt.ylabel('weighted rate for LVK network at design sensitivity')
plt.show()




### M1 more massive, M2 least massive:






In [ ]:
def obtainM1BHandM2BHassymetric(m1, m2):
    m1bh, m2bh = np.zeros_like(m1), np.zeros_like(m1)
    maskm1heavier = ( m1 >= m2)
    maskm2heavier = (m1 < m2)
    
    m1bh[maskm1heavier] = m1[maskm1heavier] 
    m1bh[maskm2heavier] = m2[maskm2heavier]
    m2bh[maskm1heavier] = m2[maskm1heavier]
    m2bh[maskm2heavier] = m1[maskm2heavier]
    
    return m1bh, m2bh # m1bh has all the heaviest systems



M_most_massive, M_least_massive = obtainM1BHandM2BHassymetric(m1=M1, m2=M2)


plt.scatter(M_most_massive, M_least_massive)
plt.xlabel('Most massive [Msun]')
plt.ylabel('Least massive [Msun]')
plt.show()



### Metallicity 

the metallicity of each data point can be obtained with "metallicitySystems"
I used a total of 53 different metallicity bins, quoted in the bottem when printing "Data.metallicityGrid" 

In [ ]:
# metallicitySystems = metallicitySystems
plt.hist(metallicitySystems, bins=100, weights=weights)
plt.xlabel('metallicity ')
plt.ylabel('weighted rate in LVK detector')
plt.show()




print('this mostly just shows my metallicity bins and where BHNS are originating from')

### Delay time  of each simulated data point in Myr

In [ ]:


plt.hist(delayTimes, bins=100, weights=weights)
plt.xlabel('Delay time [Myr] ')
plt.ylabel('weighted rate in LVK detector')
plt.show()




### TO do?: we now looked at BHNS distributions and weighted them with the LVK detector weights, how do these distributions change when instead looking at intrinsic weighted distributions? 


### try to do this by changing the weights obtained from fdata['weights_detected']  to  fdata['weights_intrinsic'] 

